In [33]:
# different statistics and descriptive plots regarding irt2 metrics

In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import irt2
from irt2.dataset import IRT2

def load_datasets():
    root = irt2.ENV.DIR.DATA / 'irt2' / 'cde'
    return {
        path.name: IRT2.from_dir(path=path) for path in (
            root / 'tiny-uniform',
            root / 'small-uniform',
            root / 'medium-uniform',
            root / 'large-uniform',
        )
    }
    # return {
    #     path.name: IRT2.from_dir(path=path) for path in (
    #         root / 'tiny-weighted',
    #         root / 'small-weighted',
    #         root / 'medium-weighted',
    #         root / 'large-weighted',
    #     )
    }



datasets = load_datasets()
for name, dataset in datasets.items():
    print(f"\nloaded {name}:")
    print(f"{dataset.graph}")
    print(f"got ratios for {len(dataset.relations)} relations")


SyntaxError: unmatched '}' (865732101.py, line 21)

In [36]:
# load source graph

import irt2
from irt2.graph import Graph
from irt2.graph import load_graph


def load_source_graph(config):
    return load_graph(
        config['graph loader'],
        config['graph name'],
        *[irt2.ENV.DIR.ROOT / path for path in config['graph loader args']],
        **{k: irt2.ENV.DIR.ROOT / path for k, path in config['graph loader kwargs'].items()},
    )


source_graph = load_source_graph(config=datasets['small-weighted'].config['create'])
print(str(source_graph))

IRT graph: [CodEx-M] (17050 entities)


In [37]:
# We want to check whether the greedy approach to subsampling actually yields
# comparable ratios between relations or if some larger bias can be spotted.
#
# (Think about having a relation "fifty" where 50% of all vertices take part and
# another "hunny" where all vertices are included. In the most extreme case, if
# by chance the greedy approach saturates "hunny" without ever encountering any
# "fifty" vertices, no vertex for "fifty" will be included in the closed world)

import numpy as np
import numpy.ma as ma

import matplotlib.pyplot as plt
from pprint import pprint

from irt2.graph import Relation
from ktz.collections import Incrementer


def get_triple_counts(source_graph, datasets):
    reldic = {'original': Relation.from_graph(g=source_graph)}
    reldic |= {name: dataset.ratios for name, dataset in datasets.items()}

    counts = np.zeros((len(reldic), len(reldic['original'])), dtype=np.uint)
    counts = ma.array(counts, mask=True)

    idxs = Incrementer()

    # start with the smallest dataset to have the most common relations
    # at the top of the count matrix

    lis = sorted(reldic.items(), key=lambda kv: len(kv[1]))
    for i, (name, rels) in enumerate(lis):
        for rel in sorted(rels, key=lambda rel: len(rel.triples), reverse=True):

            j = idxs[rel.name]
            counts[i][j] = len(rel.triples)

    ds_names = [name for (name, _) in lis]
    rel_names = [name for name in idxs.keys()]

    return ds_names, rel_names, counts


def _create_counts_table(ds_names, rel_names, counts, normalise=False):
    cols = []

    rel_ids, rel_names = zip(*map(lambda s: s.split(':'), rel_names))
    cols.append(['id'] + list(rel_ids))
    cols.append(['relation'] + [name.replace(',', '') for name in rel_names])

    if normalise:
        counts = counts / counts.sum(axis=1).reshape((1, -1)).T
        counts = counts - counts[-1]

    for name, countv in zip(ds_names, counts):
        cols.append([name] + countv.tolist())

    return list(zip(*cols))


def print_counts_csv(*args, **kwargs):
    args = get_triple_counts(*args, **kwargs)

    def csv(rows):
        rows = [
            [('' if cell is None else str(cell)) for cell in row]
            for row in rows
        ]

        rows = [','.join(row) for row in rows]
        return '\n'.join(rows)

    print('\ncounts:')
    print(csv(_create_counts_table(*args)))

    print('\nnorms:')
    print(csv(_create_counts_table(*args, normalise=True)))


print_counts_csv(source_graph=source_graph, datasets=datasets)


counts:
id,relation,tiny-weighted,small-weighted,medium-weighted,large-weighted,original
P106,occupation,1458,3217,3122,23617,71596
P27,country of citizenship,462,994,959,7524,16828
P1412,languages spoken written or signed,404,891,831,5882,12584
P30,continent,353,390,390,390,391
P19,place of birth,165,400,383,3135,7214
P495,country of origin,,1079,753,1331,2049
P140,religion,,351,296,1605,2651
P108,employer,,254,258,1831,4795
P119,place of burial,,186,128,954,1972
P159,headquarters location,,158,162,163,169
P452,industry,,16,16,16,17
P40,child,,10,14,268,391
P530,diplomatic relation,,,6225,6225,6225
P463,member of,,,4429,7296,11490
P136,genre,,,2266,6440,11761
P17,country,,,1288,1292,1323
P840,narrative location,,,586,968,1506
P69,educated at,,,571,4360,9752
P37,official language,,,402,402,403
P161,cast member,,,369,2693,9249
P264,record label,,,368,1861,3456
P172,ethnic group,,,331,1384,2293
P20,place of death,,,303,2317,5442
P1303,instrument,,,243,1730,6076
P509,cause of death,,,229